In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import os
import numpy as np

model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*(list(model.children())[:-2]))
model = torch.nn.Sequential(model, torch.nn.AdaptiveAvgPool2d((1, 1)))

model.eval()

/projectnb/cs505ws/students/bargav25/.conda/envs/lemonade/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/projectnb/cs505ws/students/bargav25/.conda/envs/lemonade/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 256

In [2]:
from tqdm import tqdm

Image.MAX_IMAGE_PIXELS = 933120000


In [3]:
test_imgs_dirpath = "/projectnb/cs640grp/materials/UBC-OCEAN_CS640/test_images_compressed_80/"

test_image_paths = os.listdir(test_imgs_dirpath)


In [5]:
test_image_paths = [os.path.join(test_imgs_dirpath, pth) for pth in test_image_paths]

In [8]:
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image
import os
import torch

class CustomDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        return image

preprocess = transforms.Compose([
    transforms.Resize((2500, 2500)),  # Resize the image
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

])



100%|██████████| 108/108 [19:47<00:00, 11.00s/it]

Feature extraction and saving completed.


In [9]:
test_feature_dir = 'test_feature_maps'

# Create test dataset and DataLoader
test_dataset = CustomDataset(test_image_paths, transform=preprocess)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Process in batches
j = 0
for batch in tqdm(test_loader):
    input_batch = batch.to(device)

    with torch.no_grad():
        features = model(input_batch)

    features = features.view(features.size(0), -1).cpu()

    # Save features
    for i, feature in enumerate(features):
        feature_path = os.path.join(test_feature_dir, test_image_paths[j].split('/')[-1][:-4] + '.pt')
        torch.save(feature, feature_path)
        j += 1

    # Clear memory
    del input_batch, features
    torch.cuda.empty_cache()

print("Feature extraction and saving completed.")

100%|██████████| 27/27 [05:06<00:00, 11.35s/it]

Feature extraction and saving completed.
